In [4]:
from abc import ABC, abstractmethod
from datetime import datetime

### Classes

In [5]:
class Transacao(ABC):
    @abstractmethod
    def registrar(self, conta):
        self.conta = conta

In [6]:
class Deposito(Transacao):
    def __init__(self, valor):
        self.__valor = valor

    @property
    def valor(self):
        return self.__valor
    
    def registrar(self, conta):
        pass

In [7]:
class Saque(Transacao):
    def __init__(self, valor):
        self.__valor = valor
    
    @property
    def valor(self):
        return self.__valor

    def registrar(self, conta):
        pass

In [8]:
class Cliente():
    def __init__(self, endereco: str):
        self.__endereco = endereco
        self.__contas = []

    def realizar_transacao(self, conta, transacao):
        pass

    def adicionar_conta(self, conta):
        pass

In [10]:
class PessoaFisica(Cliente):
    def __init__(self, endereco: str, cpf: str, nome: str, data_nascimento):
        super().__init__(endereco)
        self.__cpf = cpf
        self.__nome = nome
        self.__data_nascimento = data_nascimento

In [11]:
class Historico():
    def __init__(self, transacoes) -> None:
        self.__transacoes = []

    def adicionar_transacao(self, transacao):
        pass

In [12]:
class Conta():
    def __init__(self, numero: int) -> None:
        self.__saldo = 0
        self.__numero = numero
        self.__agencia = "0001"
        self.__cliente = Cliente()
        self.__historico = Historico()

    @property
    def saldo(self):
        return self.__saldo
    
    @property
    def numero(self):
        return self.__numero
    
    @property
    def agencia(self):
        return self.__agencia
    
    @property
    def cliente(self):
        return self.__cliente
    
    @property
    def historico(self):
        return self.__historico
    
    @classmethod
    def nova_conta(cls, numero):
        return cls(numero)
    
    def depositar(self, valor: float) -> bool:
        if valor > 0:
            self.__saldo = valor
            print(f"Seu deposito de R${valor:.2f} foi efetuado!")
        else:
            print("Nao foi possivel realizar o deposito. Tente novamente.")
            return False
        return True

    def sacar(self, valor: float) -> bool:
        pass

In [13]:
class ContaCorrente(Conta):
    def __init__(self, numero: int, limite: float, limite_saques: int) -> None:
        super().__init__(numero)
        self.__limite = limite
        self.__limite_saques = limite_saques

### Funçoes

In [14]:
def menu():
    menu = """
===================== SISTEMA BANCARIO =====================
    [d] Depositar
    [s] Sacar
    [e] Extrato
    [nc] Nova conta
    [lc] Listar contas
    [nu] Novo usuario
    [q] Sair
    => """
    return input(menu)

In [15]:
def depositar(saldo, quantia, /):
    if quantia > 0:
        saldo += quantia
        print(f"Seu deposito de R${quantia:.2f} foi efetuado!")
    else:
        print("Nao foi possivel realizar o deposito. Tente novamente.")

In [16]:
def sacar(*, saldo, quantia, numero_saques, limite_por_saque, LIMITE_SAQUES):
    regra_saldo = saldo >= quantia
    regra_limite = limite_por_saque >= quantia
    regra_saque = numero_saques < LIMITE_SAQUES
    if quantia > 0:
        if not regra_saldo:
            print("Voce nao tem saldo para realizar esta operaçao. Confira no extrato e tente novamente.")
        elif not regra_limite:
            print("Voce nao tem limite para realizar esta operaçao. Confira no extrato e tente novamente.")
        elif not regra_saque:
            print("Voce nao tem saque disponivel para realizar esta operaçao. Confira no extrato e tente novamente.")
        else:
            saldo -= quantia
            numero_saques += 1
            print(f"Seu saque de R${quantia:.2f} foi efetuado!")

In [17]:
def extrato_completo(saldo, /, *, extrato, numero_saques, LIMITE_SAQUES):
    qtd_saques = LIMITE_SAQUES - numero_saques
    cabecalhos = ["Data", "Tipo", "Valor"]
    print("===================  EXTRATO ========================")
    print(f"Seu saldo é de R${saldo:.2f}.")
    print(f"Voce tem {qtd_saques} saque(s).")
    print("=================== HISTORICO =======================")
    if extrato:
        print(f"{cabecalhos[0].center(20)} | {cabecalhos[1].center(10)} | {cabecalhos[2].center(10)}")
        for reg in extrato:
            print(f"{reg[0].center(20)} | {reg[1].center(10)} | {reg[2]}")
    else:
        print("Nao houveram movimentaçoes.")

In [18]:
def gravar_transacao(quantia, extrato):
    data = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    if opcao == "d":
        extrato.append([data, "Deposito", quantia])
    elif opcao == "s":
        extrato.append([data, "Saque", quantia])

In [19]:
def filtrar_usuarios(cpf, usuarios):
    usuario_filtrado = [usuario for usuario in usuarios if usuario["cpf"] == cpf]
    return usuario_filtrado[0] if usuario_filtrado else None

In [20]:
def criar_usuario(usuarios):

    cpf = input("Digite o CPF (somente números): ")
    usuario = filtrar_usuarios(cpf, usuarios)

    if usuario:
        print("Usuário já existe")
        return

    nome = input("Nome: ")
    data_nascimento = input("Data de nascimento: ")
    endereco = input("Endereço: (logradouro, nro - bairro - cidade/sigla estado) ")

    usuarios.append({"nome": nome, "data_nascimento": data_nascimento, "cpf": cpf, "endereco": endereco})

    print("=== Usuário criado com sucesso! ===")

In [21]:
def criar_conta_corrente(agencia, numero_conta, usuarios):

    cpf = input("Digite o CPF (somente números): ")
    usuario = filtrar_usuarios(cpf, usuarios)

    if usuario:
        print("=== Conta criada com sucesso! ===")
        return {"agencia": agencia, "numero_conta": numero_conta, "usuario": usuario}
    
    else:
        print("=== Usuário não encontrado! ===")

In [22]:
def listar_contas(contas):
    print("=================== CONTAS ========================")

    for conta in contas:
        saida = f"""
            Agencia: {conta['agencia']}
            Conta Corrente: {conta['numero_conta']}
            Titular: {conta['usuario']}
        """
        print(saida)

### Main()

In [23]:
def main():

    clientes = []
    contas = []

    while True:
        
        opcao = menu()

        if opcao == "d":
            quantia = float(input("Qual quantia deseja depositar? "))
            depositar(saldo, quantia)
            gravar_transacao(quantia, extrato)

        elif opcao == "s":
            quantia = float(input("Qual quantia deseja sacar? "))
            sacar(
                saldo = saldo, 
                quantia = quantia, 
                numero_saques = numero_saques, 
                limite_por_saque = limite_por_saque, 
                LIMITE_SAQUES = LIMITE_SAQUES
            )
            gravar_transacao(quantia, extrato)

        elif opcao == "e":
            extrato_completo(
                saldo, extrato = extrato, 
                numero_saques = numero_saques, 
                LIMITE_SAQUES = LIMITE_SAQUES
            )

        elif opcao == "nc":
            numero_conta = len(contas) + 1
            conta = criar_conta_corrente(AGENCIA, numero_conta, usuarios)

            if conta:
                contas.append(conta)

        elif opcao == "lc":
            listar_contas(contas)

        elif opcao == "nu":
            criar_usuario(usuarios)

        elif opcao == "q":
            break

        else:
            print('Operaçao invalida!')

### Teste de Funcionalidade: DEPOSITAR